In [2]:
import ctypes
import gc

In [3]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [4]:
def find_object_in_gc(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return f"Found: {obj}"
    return "Not found"

In [5]:
# define class that causes circular references
class A:
    def __init__(self):
        self.b = B(self)
        print(f'A: {id(self)} - B: {id(self.b)}')

class B:
    def __init__(self, a):
        self.a = a
        print(f'B: {id(self)} - A: {id(self.a)}')

In [6]:
# disable GC
gc.disable()

In [7]:
my_var = A()

B: 140254791448080 - A: 140254791448016
A: 140254791448016 - B: 140254791448080


In [8]:
# notice B is constructed first because it is construcuted during the __init__ of A
# and now, my_var refers to an object type A, and this object refer to an object type B, and this type B object refers back to the type A object
# this is a circular reference

In [9]:
# my_var refer to type A object
print(id(my_var))

140254791448016


In [10]:
print(id(my_var.b))

140254791448080


In [11]:
print(id(my_var.b.a))

140254791448016


In [12]:
# now let store the memory addresses of these object so that we can refer to it later
b_id = id(my_var.b)
a_id = id(my_var.b.a)
print(a_id, b_id)

140254791448016 140254791448080


In [13]:
ref_count(a_id)

2

In [14]:
ref_count(b_id)

1

In [15]:
find_object_in_gc(a_id)

'Found: <__main__.A object at 0x7f8f9d05b5d0>'

In [16]:
find_object_in_gc(b_id)

'Found: <__main__.B object at 0x7f8f9d05b610>'

In [17]:
# now, set my_var to None, that mean nothing is refer to both object A and B
my_var = None

In [18]:
ref_count(a_id)

1

In [19]:
ref_count(b_id)

1

In [20]:
# now run GC
gc.collect()

561

In [21]:
find_object_in_gc(a_id)

'Not found'

In [22]:
find_object_in_gc(b_id)

'Not found'

In [22]:
# as we see, after the GC collected, the memory addresses refer to something else or nothing at all